o1 prompt:

Write a python library of functions with a demonstation of it's usage to calculate APR from investing in shares

1. A share can be bought and sold within a day or over a longer period, months or years.

2. When a share is bought, the date and time is captured to the minute.

3. When a share is sold, the date and time is captured to the minute.

4. The application should be able to calculate APR based on a combination of sold shares
(realised profit/loss) together with the current value of shares currently held within a portfolio  

5. The application should report the realised profit/loss of shares no longer owned,
together with its bought and sold datetimes and the days elapsed between purchase and sale.
Additionally the effective APR of the share sale should be reported.

6. The application should report the unrealised profit/loss of shares still held, together
with its bought date and time. Additionally the effective APR of the share sale should be reported.

7. Within the application, all date and times should be human readable

8. The demonstation of usage should give examples of realised share dealings held for 1 day, 5 days and 6 months respectively

8. The demonstation of usage should give examples of unrealised share purchases


Respond with the code only! Nothing else!

In [ ]:
from datetime import datetime
from typing import List, Dict, Tuple

class Share:
    def __init__(self, symbol: str, purchase_price: float, quantity: int, purchase_datetime: datetime):
        self.symbol = symbol
        self.purchase_price = purchase_price
        self.quantity = quantity
        self.purchase_datetime = purchase_datetime
        self.sales: List[Tuple[datetime, float]] = []

    def sell(self, sale_datetime: datetime, sale_price: float):
        self.sales.append((sale_datetime, sale_price))

    def realized_profit_loss(self) -> Tuple[float, List[Dict]]:
        total_profit_loss = 0.0
        sale_details = []

        for sale_datetime, sale_price in self.sales:
            days_elapsed = (sale_datetime - self.purchase_datetime).days
            profit_loss = (sale_price - self.purchase_price) * self.quantity
            total_profit_loss += profit_loss

            sale_details.append({
                'sold_price': sale_price,
                'profit_loss': profit_loss,
                'sale_datetime': sale_datetime.strftime('%Y-%m-%d %H:%M'),
                'purchase_datetime': self.purchase_datetime.strftime('%Y-%m-%d %H:%M'),
                'days_elapsed': days_elapsed,
                'effective_apr': self.calculate_apr(profit_loss, days_elapsed)
            })

        return total_profit_loss, sale_details

    def calculate_apr(self, profit_loss: float, days_elapsed: int) -> float:
        if days_elapsed == 0:
            return 0.0
        apr = (profit_loss / (self.purchase_price * self.quantity)) * (365 / days_elapsed) * 100
        return apr

    def unrealized_profit_loss(self, current_price: float) -> Dict:
        unrealized_profit_loss = (current_price - self.purchase_price) * self.quantity
        return {
            'unrealized_profit_loss': unrealized_profit_loss,
            'purchase_datetime': self.purchase_datetime.strftime('%Y-%m-%d %H:%M'),
            'effective_apr': self.calculate_apr(unrealized_profit_loss, (datetime.now() - self.purchase_datetime).days)
        }


class Portfolio:
    def __init__(self):
        self.shares: List[Share] = []

    def add_share(self, share: Share):
        self.shares.append(share)

    def calculate_realized_and_unrealized(self, current_prices: Dict[str, float]) -> None:
        for share in self.shares:
            if share.sales:
                realized_pl, sale_details = share.realized_profit_loss()
                print(f'Share: {share.symbol}, Realized Profit/Loss: {realized_pl}, Details: {sale_details}')
            else:
                if share.symbol in current_prices:
                    unrealized_info = share.unrealized_profit_loss(current_prices[share.symbol])
                    print(f'Share: {share.symbol}, Unrealized Profit/Loss: {unrealized_info}')


# Demonstration of usage
if __name__ == "__main__":
    portfolio = Portfolio()

    # Realized shares
    share1 = Share("AAPL", 150.0, 10, datetime(2023, 10, 1, 10, 0))  # Bought
    share1.sell(datetime(2023, 10, 2, 10, 0), 155.0)  # Sold after 1 day

    share2 = Share("AAPL", 150.0, 10, datetime(2023, 9, 1, 10, 0))  # Bought
    share2.sell(datetime(2023, 9, 6, 10, 0), 160.0)  # Sold after 5 days

    share3 = Share("TSLA", 700.0, 5, datetime(2023, 4, 1, 10, 0))  # Bought
    share3.sell(datetime(2023, 10, 1, 10, 0), 900.0)  # Sold after 6 months

    portfolio.add_share(share1)
    portfolio.add_share(share2)
    portfolio.add_share(share3)

    # Unrealized shares
    share4 = Share("GOOGL", 2500.0, 2, datetime(2023, 9, 10, 10, 0))  # Bought
    portfolio.add_share(share4)

    current_prices = {
        "AAPL": 160.0,
        "TSLA": 850.0,
        "GOOGL": 2700.0
    }

    portfolio.calculate_realized_and_unrealized(current_prices)


Share: AAPL, Realized Profit/Loss: 50.0, Details: [{'sold_price': 155.0, 'profit_loss': 50.0, 'sale_datetime': '2023-10-02 10:00', 'purchase_datetime': '2023-10-01 10:00', 'days_elapsed': 1, 'effective_apr': 1216.6666666666665}]
Share: AAPL, Realized Profit/Loss: 100.0, Details: [{'sold_price': 160.0, 'profit_loss': 100.0, 'sale_datetime': '2023-09-06 10:00', 'purchase_datetime': '2023-09-01 10:00', 'days_elapsed': 5, 'effective_apr': 486.66666666666663}]
Share: TSLA, Realized Profit/Loss: 1000.0, Details: [{'sold_price': 900.0, 'profit_loss': 1000.0, 'sale_datetime': '2023-10-01 10:00', 'purchase_datetime': '2023-04-01 10:00', 'days_elapsed': 183, 'effective_apr': 56.986729117876656}]
Share: GOOGL, Unrealized Profit/Loss: {'unrealized_profit_loss': 400.0, 'purchase_datetime': '2023-09-10 10:00', 'effective_apr': 6.252676659528908}
